In [ ]:
import pandas as pd
import json
import os

import sys
sys.path.append('../')

import dft_descriptors.numbering_CO as nb
from rdkit import Chem
from dft_descriptors.dft_featurize import *
from dft_descriptors.prepocessing import preprocess

from aqc_utils.molecule import molecule
from aqc_utils.db_functions import *
from aqc_utils.openbabel_functions import *

import hashlib
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
df = pd.read_csv("../data_csv/NiCOlit.csv", sep = ',')
df = df[df["Reactant Smile (C-O)"].isna() == False]

In [ ]:
df = preprocess(df)
df = df[df["Mechanism"] != 'Review']

In [ ]:
#Need to create to tables with numbered AL and numbered Bases by "shared atoms"

In [ ]:
mol_df = db_select_molecules()
#mol_df.to_csv("../data_csv/mol_df.csv", sep=',',index=False)
#mol_df = pd.read_csv("../data_csv/mol_df.csv", sep=',')

In [ ]:
sub_df = generates_descriptors(mol_df, "LewisAcid")

In [ ]:
df = pd.read_csv('../data_csv/AL_dft.csv', sep = ',', index_col=0)
df

In [ ]:
df = df.append(sub_df)
df = df[df.duplicated(keep = 'first') == False]

In [ ]:
df

In [ ]:
#df.to_csv('../data_csv/AL_dft.csv', sep=',', index=True)

In [ ]:
df = pd.read_csv('../data_csv/ligand_dft.csv', sep = ',', index_col=0)
idx_to_drop = []
i = 0
for smi in df.index.to_list():
    if smi in sub_df.index.to_list():
        idx_to_drop.append(smi)
    i+=1

#idx_to_drop.append("CC(N1c2ccccc2N(C1CC1N(C)c2c(N1C(C)C)cccc2)C)C")
#idx_to_drop.append("CN1C(CC2N(C(C)C)c3c(N2C(C)C)cccc3)N(c2c1cccc2)C")
df = df.drop(labels=idx_to_drop, axis=0)

In [ ]:
df = df.append(sub_df)
#df.to_csv('../data_csv/ligand_dft.csv', sep=',', index=True)

In [ ]:
df

In [ ]:
df = df.append(sub_df)
df = df[df.duplicated(keep='first')== False]
df

In [ ]:
#sub_df.to_csv('../data_csv/AX_dft.csv', index= True)

In [ ]:
df = df.append(sub_df)
df = df[df.duplicated(keep='first')== False]
df

In [ ]:
df.to_csv('../data_csv/ligand_dft.csv', sep=',', index=True)

In [ ]:
dft_set = {'gaussian_config': {'theory': 'b3lyp',
                               'light_basis_set': '6-31G*',
                               'heavy_basis_set': 'LANL2DZ',
                               'generic_basis_set': 'genecp',
                               'max_light_atomic_number': 36},
               'gaussian_tasks': ['opt b3lyp/6-31G* scf=(xqc,tight)',
                                  'freq b3lyp/6-31G* volume NMR pop=NPA density=current Geom=AllCheck Guess=Read',
                                  'TD(NStates=10, Root=1) b3lyp/6-31G* volume pop=NPA density=current Geom=AllCheck Guess=Read'],
               'max_num_conformers': 1,
               'class': '',
               'subclass': '',
               'type': '',
               'subtype': ''}

In [ ]:
dft_set['gaussian_config']

In [ ]:
data_df = pd.read_csv('../data_csv/Data_test11032021.csv', sep = ',') 
data_df = pp.preprocess(data_df)
data_df = data_df[data_df['Ligand effectif'].notna()]
data_df = data_df[data_df['Ligand effectif'] != 'NoLigand']
unik_lig = np.unique(data_df["Ligand effectif"])

In [ ]:
can_smis = np.unique([Chem.CanonSmiles(smi) for smi in unik_lig])